In [1]:
# import findspark
# findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 10000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

3.140552


In [2]:
pklotsdf = sqlContext.read.format('csv').options(header='true', "timestampFormat","yyyy/MM/dd HH:mm:ss.SSS").load('../testing/data/Lot1North.csv')

SyntaxError: positional argument follows keyword argument (<ipython-input-2-c5791d1bc8b0>, line 1)

In [46]:
!pwd

/Users/kmurali/Cloud/CloudComputing


In [47]:
from IPython.display import display
pklotsdf.show(n=100)

+--------------------+-----------+-----------+--------------------+-----+---------+-----------+---------+--------------------+
|           Date/Time|        Lot|Description|             Address|  Zip| Latitude|  Longitude|Available|           Map Point|
+--------------------+-----------+-----------+--------------------+-----+---------+-----------+---------+--------------------+
|11/07/2017 06:10:...|Lot 1 North| Off of PCH|1550 Pacific Coas...|90401|34.010806|-118.497361|      592|(34.010806, -118....|
|11/07/2017 06:40:...|Lot 1 North| Off of PCH|1550 Pacific Coas...|90401|34.010806|-118.497361|      639|(34.010806, -118....|
|11/07/2017 06:25:...|Lot 1 North| Off of PCH|1550 Pacific Coas...|90401|34.010806|-118.497361|      623|(34.010806, -118....|
|11/07/2017 06:20:...|Lot 1 North| Off of PCH|1550 Pacific Coas...|90401|34.010806|-118.497361|      614|(34.010806, -118....|
|11/07/2017 07:00:...|Lot 1 North| Off of PCH|1550 Pacific Coas...|90401|34.010806|-118.497361|      662|(34.01

In [51]:
pklotsdf.sort("Date/Time")

DataFrame[Date/Time: string, Lot: string, Description: string, Address: string, Zip: string, Latitude: string, Longitude: string, Available: string, Map Point: string]

In [123]:
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import monotonically_increasing_id
import pyspark.sql.functions as func
from pyspark.sql.window import Window

#Refer to pattern for timestamps: https://docs.oracle.com/javase/8/docs/api/java/text/SimpleDateFormat.html

pattern = 'MM/dd/yyyy hh:mm:ss aa'

df = pklotsdf.withColumn("DateTimeStamp", unix_timestamp(pklotsdf["Date/Time"], pattern).cast(
        "timestamp"))

#df.show(n=100)
#new_df = df.orderBy("DateTimeStamp")

my_window = Window.orderBy("DateTimeStamp")

#df2 = new_df.withColumn("id", monotonically_increasing_id())
df2 = new_df.withColumn("id", func.row_number().over(my_window))
df2.show(n=1000)






+--------------------+-----------+-----------+--------------------+-----+---------+-----------+---------+--------------------+-------------------+----+
|           Date/Time|        Lot|Description|             Address|  Zip| Latitude|  Longitude|Available|           Map Point|      DateTimeStamp|  id|
+--------------------+-----------+-----------+--------------------+-----+---------+-----------+---------+--------------------+-------------------+----+
|11/01/2017 12:00:...|Lot 1 North| Off of PCH|1550 Pacific Coas...|90401|34.010806|-118.497361|      921|(34.010806, -118....|2017-11-01 00:00:00|   1|
|11/01/2017 12:05:...|Lot 1 North| Off of PCH|1550 Pacific Coas...|90401|34.010806|-118.497361|      923|(34.010806, -118....|2017-11-01 00:05:00|   2|
|11/01/2017 12:10:...|Lot 1 North| Off of PCH|1550 Pacific Coas...|90401|34.010806|-118.497361|      922|(34.010806, -118....|2017-11-01 00:10:00|   3|
|11/01/2017 12:15:...|Lot 1 North| Off of PCH|1550 Pacific Coas...|90401|34.010806|-118.

In [125]:
import pyspark.sql.functions as func
from pyspark.sql import Window

#Assuming we don't have data from immediately prior, I'm only going to be able to use prior weeks 
# in order to make predictions
num_intervals_in_one_day= int(24 * 60 / 5)
num_intervals_in_one_day_offset_hour = int(24 * 60 / 5) - 12
num_intervals_15_min = int(15/5)
num_intervals_30_min = int(30/5)
num_intervals_45_min = int(45/5)
num_intervals_60_min = int(60/5)
num_intervals_120_min = int(120/5)
num_intervals_180_min = int(180/5)
num_intervals_240_min = int(240/5)

intervals = [num_intervals_in_one_day, num_intervals_in_one_day_offset_hour, num_intervals_15_min, num_intervals_30_min,
            num_intervals_60_min, num_intervals_60_min, num_intervals_120_min, 
            num_intervals_180_min, num_intervals_240_min]


my_window = Window.partitionBy().orderBy("id")

df3 = df2.select('Available', 'DateTimeStamp', "id")
#df3.show(n=1000)

for interval in intervals:
    col_name = str(interval*5) + 'min interval'
    col_time_stamp_name = str(interval*5) + 'min interval timestamp'
    df4 = df3.withColumn(col_name, func.lag(df3["Available"], 
            count = interval, default=None).over(my_window)).withColumn(col_time_stamp_name,
            func.lag(df2["DateTimeStamp"], count = interval, default=None).over(my_window))
    df3 = df4 
    

#df4.show(n=1000)

df5 = df4.where(df4['id'] >= 300)
df5.show(n=1000)


+---------+-------------------+----+----------------+--------------------------+----------------+--------------------------+--------------+------------------------+--------------+------------------------+--------------+------------------------+---------------+-------------------------+---------------+-------------------------+---------------+-------------------------+
|Available|      DateTimeStamp|  id|1440min interval|1440min interval timestamp|1380min interval|1380min interval timestamp|15min interval|15min interval timestamp|30min interval|30min interval timestamp|60min interval|60min interval timestamp|120min interval|120min interval timestamp|180min interval|180min interval timestamp|240min interval|240min interval timestamp|
+---------+-------------------+----+----------------+--------------------------+----------------+--------------------------+--------------+------------------------+--------------+------------------------+--------------+------------------------+--------------

In [149]:
import pyspark.mllib
import pyspark.mllib.regression
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import *
print (new_df.schema)

temp_features = df5.rdd.map(lambda line:LabeledPoint(float(line[0]),
                    [float(line[3]),float(line[5]),float(line[7]), float(line[9]), float(line[11]), 
                     float(line[13]) ,float(line[15]), float(line[17])])).cache()



#lag(new_df[], count=1, default=None)

StructType(List(StructField(Date/Time,StringType,true),StructField(Lot,StringType,true),StructField(Description,StringType,true),StructField(Address,StringType,true),StructField(Zip,StringType,true),StructField(Latitude,StringType,true),StructField(Longitude,StringType,true),StructField(Available,StringType,true),StructField(Map Point,StringType,true),StructField(DateTimeStamp,TimestampType,true)))


In [197]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

#print(temp_features.take(300))

model = LinearRegressionWithSGD.train(temp_features, iterations=100000, step=0.000003)

valuesAndPreds = temp_features.map(lambda p: (p.label, model.predict(p.features)))

MSE = valuesAndPreds \
    .map(lambda vp: (vp[0] - vp[1])**2) \
    .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))
#https://www.arundhaj.com/blog/calculate-difference-with-previous-row-in-pyspark.html

print(model.weights)
#Have to calculate offset in row values

#

Mean Squared Error = 2922.0266294064622
[0.5150444786386184,0.4824431304524283]


Mean Squared Error = 1673.8845170635495
[-0.12432401261030118,0.057220493748445164,0.1962419888127126,0.47459642319120693,0.5574871684794405,0.20011490152627415,-0.34711828882394785,-0.020972052292898127]

- Results indicate that some of the inputs in each training vector are entirely useless, they have negative wieghts associated with them.  



In [198]:
import pyspark.mllib
import pyspark.mllib.regression
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import *
print (new_df.schema)

temp_features = df5.rdd.map(lambda line:LabeledPoint(float(line[0]),
                    [float(line[3]),float(line[5]),float(line[7]), float(line[9]), float(line[11]), 
                     float(line[13])])).cache()

StructType(List(StructField(Date/Time,StringType,true),StructField(Lot,StringType,true),StructField(Description,StringType,true),StructField(Address,StringType,true),StructField(Zip,StringType,true),StructField(Latitude,StringType,true),StructField(Longitude,StringType,true),StructField(Available,StringType,true),StructField(Map Point,StringType,true),StructField(DateTimeStamp,TimestampType,true)))


In [199]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

#print(temp_features.take(300))

model = LinearRegressionWithSGD.train(temp_features, iterations=100000, step=0.0000038)

valuesAndPreds = temp_features.map(lambda p: (p.label, model.predict(p.features)))

MSE = valuesAndPreds \
    .map(lambda vp: (vp[0] - vp[1])**2) \
    .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))
#https://www.arundhaj.com/blog/calculate-difference-with-previous-row-in-pyspark.html

print(model.weights)
#Have to calculate offset in row values

#

Mean Squared Error = 1441.312370932672
[-0.03290916184627791,0.13161216676873744,0.3117676625222694,0.36398101479123124,0.19093216325923634,0.034380817123766276]


Slightly better performance 
Mean Squared Error = 1441.312370932672
[-0.03290916184627791,0.13161216676873744,0.3117676625222694,0.36398101479123124,0.19093216325923634,0.034380817123766276]

With the following parameters

iterations=100000, step=0.0000038), when the number of features was cut down

In [182]:
import pyspark.mllib
import pyspark.mllib.regression
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import *
print (new_df.schema)

temp_features = df5.rdd.map(lambda line:LabeledPoint(float(line[0]),
                    [ float(line[9]), float(line[11])])).cache()



#lag(new_df[], count=1, default=None)

StructType(List(StructField(Date/Time,StringType,true),StructField(Lot,StringType,true),StructField(Description,StringType,true),StructField(Address,StringType,true),StructField(Zip,StringType,true),StructField(Latitude,StringType,true),StructField(Longitude,StringType,true),StructField(Available,StringType,true),StructField(Map Point,StringType,true),StructField(DateTimeStamp,TimestampType,true)))


In [196]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD, LinearRegressionModel

#print(temp_features.take(300))

model = LinearRegressionWithSGD.train(temp_features, iterations=10000, step=0.000004)

valuesAndPreds = temp_features.map(lambda p: (p.label, model.predict(p.features)))

MSE = valuesAndPreds \
    .map(lambda vp: (vp[0] - vp[1])**2) \
    .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))
#https://www.arundhaj.com/blog/calculate-difference-with-previous-row-in-pyspark.html

print(model.weights)
#Have to calculate offset in row values

#

[LabeledPoint(967.0, [963.0,952.0]), LabeledPoint(966.0, [966.0,953.0]), LabeledPoint(967.0, [963.0,957.0]), LabeledPoint(969.0, [962.0,959.0]), LabeledPoint(970.0, [963.0,959.0]), LabeledPoint(972.0, [966.0,963.0]), LabeledPoint(972.0, [967.0,963.0]), LabeledPoint(974.0, [966.0,966.0]), LabeledPoint(974.0, [967.0,963.0]), LabeledPoint(975.0, [969.0,962.0]), LabeledPoint(975.0, [970.0,963.0]), LabeledPoint(977.0, [972.0,966.0]), LabeledPoint(979.0, [972.0,967.0]), LabeledPoint(979.0, [974.0,966.0]), LabeledPoint(980.0, [974.0,967.0]), LabeledPoint(979.0, [975.0,969.0]), LabeledPoint(978.0, [975.0,970.0]), LabeledPoint(978.0, [977.0,972.0]), LabeledPoint(976.0, [979.0,972.0]), LabeledPoint(975.0, [979.0,974.0]), LabeledPoint(977.0, [980.0,974.0]), LabeledPoint(975.0, [979.0,975.0]), LabeledPoint(976.0, [978.0,975.0]), LabeledPoint(976.0, [978.0,977.0]), LabeledPoint(974.0, [976.0,979.0]), LabeledPoint(974.0, [975.0,979.0]), LabeledPoint(975.0, [977.0,980.0]), LabeledPoint(974.0, [975.0,

Next Steps: The timestamps would be a very valuable feature to add. 

We need more complex non-linear relationships between features like the time, and the predicted parking rate change - for example, parking will peak in afternoon, and decrease  from them on 

MultilayerPerceptronClassifier